In [1]:
import pywikibot
import collections
from util import *
from tqdm import tqdm
import random

In [2]:
# Store things as pairs: (queue, dict) where queue is
# the queue associated to a bfs, and dict is
# a dictionary sending each neighbor of a previous element of queue
# to its parent that showed up earliest in the list.
# A possible future update is to also include the distance
# from the origin in the dict, so the image of the dict
# would be an ordered pair (parent, distance).
# Another potential improvement is to expand the breadth-first search
# on the side whose dictionary is smaller, rather than just alternating

In [3]:
# Updates a pair and returns 1 if it finds a path,
# 0 if it does not find a path but updates successfully,
# and -1 if there is nowhere to expand.
# By default, the first pair is the one updated.
# careful_bookkeeping means that the pairs will be updated correctly
# even if a path is found.
def expand_one_node(pair1, pair2, expand_first = True, pair1_to_pair2 = True, careful_bookkeeping=True):
    if not expand_first:
        return expand_one_side(pair2, pair1, pair1_to_pair2 = not pair1_to_pair2, careful_bookkeeping = careful_bookkeeping)
    
    try:
        item_to_expand, path = pair1[0].popleft()
        #print(item_to_expand)

        if pair1_to_pair2:
            potential_expansions = item_to_expand.get_successors()
        else:
            potential_expansions = item_to_expand.get_predecessors()
            
        # good potential expansions are those that have not been seen before
        # in either dictionary.  It is not completely obvious, but not hard to show
        # that if we get stuck by avoiding words in this way, then there is no path
        # from start to end.
        good_potential_expansions = []
        for item in potential_expansions:  
            if valid_page(item.title()):
                if not careful_bookkeeping:
                    if item in pair2[1].keys():
                        # if we have seen this item before when expanding the other pair
                        pair1[0].append((item, path + [item]))
                        return 1
                    elif item not in pair1[1].keys():
                        good_potential_expansions.append((item, path + [item]))
                        pair1[1][item] = item_to_expand
                else:
                    pass #TODO
        
        pair1[0].extend(good_potential_expansions)
        
    except:
        # If the queue is empty.
        #
        # In the case of BFS, this case should never occur
        # because empty queues are caught in expand_one_side()
        # and moreover, if the queue is empty, the condition in
        # the for loop in expand_one_side() is never true, so
        # this method never gets called anyway.
        return -1


In [4]:
# Expands every node in one pair
# Which pair depends on the value of expand_first
# which direction to expand (linksfrom or backlinks) depends on pair1_to_pair2
#
# Returns an ordered pair (result, num_expansions)
# where result indicates whether a path was found
# and num_expansions indicates how many nodes were expanded
def expand_one_side(pair1, pair2, expand_first = True, pair1_to_pair2 = True, careful_bookkeeping=True):    
    if expand_first:
        num_to_expand = len(pair1[0])
    else:
        num_to_expand = len(pair2[0])
    
    # This if condition could be moved to below the for loop
    # and it might be marginally faster, but I think
    # it's better style to put it here
    if num_to_expand == 0:
        return (-1,0)
    
    for i in range(num_to_expand):
        result = expand_one_node(pair1, pair2, expand_first, pair1_to_pair2, careful_bookkeeping)
        if result != None:
            return (result,i+1)
        
    return (0,num_to_expand)
        

In [5]:
# use_first means that the path works by
# travelling down the queue of pair1
# followed by going backwards down pair2;
# not use_first means the opposite.
def generate_path(pair1,pair2,use_first):
    if use_first:
        endpoint, path = pair1[0].pop()
        dict = pair2[1]
        while endpoint != None:
            endpoint = dict[endpoint]
            path.append(endpoint)
        path = path[:-1]
    else:
        # This is equivalent to the following:
        # path = generate_path(pair2,pair1,not use_first)
        # path.reverse()
        endpoint, path = pair2[0].pop()
        dict = pair1[1]
        while endpoint != None:
            endpoint = dict[endpoint]
            path.append(endpoint)
        path = path[:-1]
        path.reverse()
    return path

In [6]:
def set_use_first(previous_use_first, pair1, pair2):
    return not previous_use_first

In [7]:
# if return_expansions == False, the output is the path as a list
# if return_expansions == True, the output is a tuple (path, num_expansions)
# where num_epansions is the number of nodes expanded.
# In either case, the number of nodes expanded is counted;
# return_expansions only affects whether it is returned
def bidirectional_BFS_graph(start, end, return_expansions=False):
    queue1 = collections.deque()
    queue2 = collections.deque()
    num_expansions = 0
    
    queue1.append((start,[start]))
    queue2.append((end,[end]))
    dict1 = {start:None}
    dict2 = {end:None}

    use_first = True
    while True:
        result = expand_one_side((queue1, dict1), (queue2, dict2), expand_first = use_first, careful_bookkeeping = False)
        num_expansions += result[1]
        if result[0] == 1:
            path = generate_path((queue1,dict1),(queue2,dict2),use_first)
            if return_expansions:
                return (path,num_expansions)
            else:
                return path
        elif result[0] == -1:
            if return_expansions:
                return (False, num_expansions)
            else:
                return False
        
        use_first = set_use_first(use_first, (queue1, dict1), (queue2, dict2))

In [8]:
class PageNode:
    def __init__(self,page):
        self.page = page
        
    def __str__(self):
        return str(self.page)
    
    def __repr__(self):
        return repr(self.page)
    
    def __eq__(self, other):
        try:
            return self.page == other.page
        except:
            # If other does not have a page attribute
            # (Main intended use case: If other == None)
            return False
    
    def __ne__(self, other):
        return not self == other
    
    def __hash__(self):
        return hash(self.page)
    
    def title(self):
        return self.page.title()
        
    def get_successors(self):
        pages = [PageNode(page) for page in list(self.page.linkedPages())]
        random.shuffle(pages)
        return pages
    
    def get_predecessors(self):
        pages = [PageNode(page) for page in list(self.page.backlinks())]
        random.shuffle(pages)
        return pages

In [9]:
site = pywikibot.Site("en", "wikipedia")

In [10]:
def PageNode_from_title(title):
    return PageNode(pywikibot.Page(site,title))

In [11]:
def bidirectional_BFS(start, end):
    return bidirectional_BFS_graph(PageNode_from_title(start),PageNode_from_title(end), return_expansions=True)

In [12]:
examples = get_samples(100)

In [13]:
def run_search(search_method, search_tuples, **kwargs):
    results = {}
    for start, goal in tqdm(search_tuples):
        results[(start, goal)] = search_method(start, goal, **kwargs)
    return results

In [14]:
output = run_search(bidirectional_BFS, examples)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [28:55<00:00, 11.51s/it]


In [15]:
output

{('Transhumanism',
  'Saturn'): ([Page('Transhumanism'),
   Page('Volcanic winter'),
   Page('Sun'),
   Page('Saturn')], 6),
 ('Vacuum',
  'Jim Henson'): ([Page('Vacuum'),
   Page('Incandescent light bulb'),
   Page('Easy-Bake Oven'),
   Page('Jim Henson')], 105),
 ('Earth',
  'Chimpanzee'): ([Page('Earth'),
   Page('History of the world'),
   Page('Technology'),
   Page('Chimpanzee')], 5),
 ('Renaissance',
  'Dancing with the Stars'): ([Page('Renaissance'),
   Page('Italians'),
   Page('Gianni Rivera'),
   Page('Dancing with the Stars')], 17),
 ('dna',
  'War of 1812'): ([Page('Dna'),
   Page('DNA'),
   Page('Agriculture'),
   Page('Industrial Revolution'),
   Page('War of 1812')], 8),
 ('Kidney',
  'Wesley Snipes'): ([Page('Kidney'),
   Page('Latin'),
   Page('Proto-Indo-European language'),
   Page('Michael Fassbender'),
   Page('Wesley Snipes')], 449),
 ('Brown Bear',
  'World Series'): ([Page('Brown Bear'),
   Page('Brown bear'),
   Page('Galápagos sea lion'),
   Page('Colombia'),